In [1]:
### 라이브러리 정의하기
# - 모델 또는 파일 저장 라이브러리
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv("D:/1조프로젝트/Voice_Phishing/call_type1.csv", encoding = 'cp949')

In [3]:
### 결측치 처리하기
# 결측치가 있는 모든 행 삭제하기
df = df.dropna(how = "any")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      500 non-null    object
 1   phishing  500 non-null    int64 
 2   type      500 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 11.8+ KB


In [4]:
### 훈련 및 테스트 데이터 읽어들이기
train_data = df[:370]
test_data = df[370:]
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      370 non-null    object
 1   phishing  370 non-null    int64 
 2   type      370 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 370 to 499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      130 non-null    object
 1   phishing  130 non-null    int64 
 2   type      130 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.2+ KB


In [5]:
### 데이터 정제하기
# - 중복 데이터의 갯수 확인하기
# - 중복되는 데이터가 있다면 duplicate으로 중복 데이터 처리하기
train_data["text"].nunique()

369

In [6]:
### 중복데이터 제거하기
## inplace = True : 별도의 변수에 넣지 않아도 바로 반영됨
train_data.drop_duplicates(subset=["text"], inplace = True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14124\974164053.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop_duplicates(subset=["text"], inplace = True)


In [7]:
len(train_data)

369

In [8]:
### 데이터 정제하기
# - 한글과 공백을 제외하고 모두 제거하기
train_data["text"] = \
    train_data["text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
train_data

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14124\3030497016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["text"] = \


,text,phishing,type
0,제가 답답해 죽겠네 지금 선생님 본인이래요 자기가 네 그 성함이 아니에요 그 분이 ...,1,1
1,내일 출근입니다 언제 받으셨어요 문자는 언제 받으셨으니 좀 전에 받으셨어요네 성함이...,1,1
2,금융범죄 같아서 전화 드렸습니다 네 어떤 상황이신가요 제가 장사를 하...,0,1
3,네 두 가지 방법이 있는데 문제는 국민은행에 통장을 하나 신설 하시는 거예요 드시고...,1,1
4,안녕하세요 문의 드릴게 있어서요 네 무엇을 도와드릴까요 제가 오늘 중...,0,1
...,...,...,...
365,여보세요 네 여보세요 네 네 안녕하세요 고객님 케이비 국민 은행 김진아 대리입니다 ...,1,1
366,여보세요 여보세요 네네 아 네 고객님 결과 나오셨구요 네네 아 지금 저희 쪽 한도는...,1,1
367,여보세요 네 ㅇㅇ고객님 되시죠 네 안녕하세요 저희 캐피탈입니다 고객님 네 지금 전화...,1,1
368,제가 가입한 상품에서 보장이 안되는 게 있다고 들었는데 확인가능한가요 년도 ...,0,1


In [9]:
## (for문을 많이 돌려야되므로 데이터프레임에서 처리하기)
### 특정 행에 공백(스페이스 한 칸)이 있는 데이터 조회해서
# NaN(null)으로 변환한 뒤에
# null인 행 삭제 함수를 사용해서 삭제시키기
### 공백이 있는 행을 찾기
train_data["text"] = \
        train_data["text"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
train_data["text"].replace("", np.nan, inplace = True)

### 결측치가 있다면, 행 제거하기
train_data = train_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
train_data.isnull().sum()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14124\1933419508.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["text"] = \
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14124\1933419508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["text"].replace("", np.nan, inplace = True)


text        0
phishing    0
type        0
dtype: int64

In [10]:
### 단어 토큰화
# - 불용어 처리 동시에 진행
okt = Okt()

In [11]:
### 불용어 정의하기
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해'],
 ['을'],
 ['를'],
 ['에'],
 ['의'],
 ['가'],
 ['으로'],
 ['로'],
 ['에게'],
 ['뿐이다'],
 ['의거하여'],
 ['근거하여'],
 ['입각하여'],
 ['기준으로'],
 ['예하면'],
 ['예를 들면'],
 ['예를 들자면'],
 ['저'],
 ['소인'],
 ['소생'],
 ['저희'],
 ['지말고'],
 ['하지마'],
 ['하지마라'],
 ['다른'],
 ['물론'],
 ['또한'],
 ['그리고'],
 ['비길수 없다'],
 ['해서는 안된다'],
 ['뿐만 아니라'],
 ['만이 아니다'],
 ['만은 아니다'],
 ['막론하고'],
 ['관계없이'],
 ['그치지 않다'],
 ['그러나'],
 ['그런데'],
 ['하지만'],
 ['든간에'],
 ['논하지 않다'],
 ['따지지 않다'],
 ['설사'],
 ['비록'],
 ['더라도'],
 ['아니면'],
 ['만 못하다'],
 ['하는 편이 낫다'],
 ['불문하고'],
 ['향하여'],
 ['향해서'],
 ['향하다'],
 ['쪽으로'],
 ['틈타'],
 ['이용하여'],
 ['타다'],
 ['오르다'],
 ['제외하고'],
 ['이 외에'],
 ['이 밖에'],
 ['하여야'],
 ['비로소'],
 ['한다면 몰라도'],
 ['외에도'],
 ['이곳'],
 ['여기'],
 ['부터'],
 ['기점으로'],
 ['따라서'],
 ['할 생각이다'],
 ['하려고하다'],
 ['이리하여'],
 ['그리하여'],
 ['그렇게 함으로써'],
 ['하지만'],
 ['일때'],
 ['할때'],
 ['앞에서'],
 ['중에서'],
 ['보는데서'],
 ['으로써'],
 ['로써'],
 ['까지'],
 ['해야한다'],
 ['일것이다'],
 ['반드시'],
 ['할줄알다'],
 ['할수있다'],
 [

In [12]:
### 단어 토큰화
# - 훈련데이터
X_train = []
for sentence in tqdm(train_data["text"]) :
    # print(sentence)
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem = True)
    # 불용어 처리
    stopwords_removed_sentence = [word
                                     for word in tokenized_sentence
                                         if not word in stopwords]
    # 각 문장별 단어 토큰화 결과 리스트에 추가하기
    X_train.append(stopwords_removed_sentence)
    
print(X_train)

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:22<00:00, 16.37it/s]

[['제', '가', '답답하다', '죽겠네', '지금', '선생님', '본인', '이래', '요', '자기', '가', '네', '그', '성함', '이', '아니다', '그', '분', '이', '아니다', '은행', '을', '통해', '서', '통화', '하다', '답답하다', '진짜', '오다', '제', '가', '은행', '아니다', '결과', '는', '보시', '면', '알다', '거', '예요', '지금', '어떻다', '사태', '가', '지금', '발생', '을', '하다', '복잡하다', '돼다', '지금', '굉장하다', '복잡하다', '경찰', '에서', '알다', '하다', '일이구', '요', '저', '도', '경찰서', '에', '신고', '를', '하다', '제', '가', '편취', '를', '하다', '제', '가', '벌', '을', '받다', '그', '쪽', '에서', '잘못', '을', '하다', '그', '쪽', '에서', '벌', '을', '받다', '그런데', '이', '거', '를', '뭐', '그거', '는', '네', '그렇다', '사람', '을', '이렇다', '식', '으로', '완전', '개떡', '만들다', '놓다', '아', '진짜', '이렇다', '사기', '가', '없다', '아니다', '지금', '전', '지금', '납득', '이', '안', '되다', '납득', '이', '지금', '저희', '어떻다', '돌아가다', '모르다', '지금', '고객', '님', '이', '하다', '말씀', '이', '전부', '거짓', '이', '예요', '지금', '제', '가', '무슨', '거짓', '이란', '말씀', '이세', '요', '지금', '상황', '이', '그', '상황', '이', '아니다', '지금', '무슨', '상황', '을', '다시', '한번', '말씀', '하다', '보다', '통화', '를', '하다', '적', '이', '없다', '시대', '요', '통화', '한적'

In [13]:
### 테스트 데이터 정제 작업 먼저 진행
# - 한글과 공백을 제외하고 모두 제거하기
test_data["text"] = \
    test_data["text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
test_data

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14124\3935041760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["text"] = \


,text,phishing,type
370,여보세요 예 고객님 롯데 심사팀이에요 예 예 예 아까 좀 전에 전화 드렸는데 전화 ...,1,1
371,제가 현금카드를 분실했는데요 네 분실신고를 접수해드릴까요 아뇨 어제 ...,0,1
372,그래서 지금 대출 자금 받아서 입금해 주라 이것만 약속을 해주세요 저도 왜냐면은 여...,1,1
373,요번에 유방에서 섬유선종 제거를 했어요 수술진단금을 받을 수 있나요 ...,0,1
374,예 고객님 농협 캐피탈인데요 어제 대출 조건이 좀 완화가 되서 연락 드렸는데 혹시 ...,1,1
...,...,...,...
495,수표를 분실신고할 때 비용이 든다는데 맞나요 네 맞습니다 어떤 비용이 드...,0,1
496,현금카드를 분실했습니다 네 분실신고 접수 도와드리겠습니다 네 현금...,0,1
497,여보세요 예 고객님 맞으신가요 네 예 현대 캐피탈 송금과 이진욱 과장입니다 피해자...,1,1
498,반갑습니다 대학교 입니다 은행의 이체 과정을 알고 싶은데요 어떤 매체를 이용...,0,1


In [14]:
### 특정 행에 공백(스페이스 한 칸)이 있는 데이터 조회해서
# NaN(null)으로 변환한 뒤에
# null인 행 삭제 함수를 사용해서 삭제시키기
### 공백이 있는 행을 찾기
test_data["text"] = \
        test_data["text"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
test_data["text"].replace("", np.nan, inplace=True)

### 결측치가 있다면, 행 제거하기
test_data = test_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
test_data.isnull().sum()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14124\2935756037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["text"] = \
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14124\2935756037.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["text"].replace("", np.nan, inplace=True)


text        0
phishing    0
type        0
dtype: int64

In [15]:
### 단어 토큰화
# - 테스트 데이터
## 공백이 있으면 안되므로 확인하기
X_test = []
for sentence in tqdm(test_data["text"]) :
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem = True)
    # 불용어 처리
    stopwords_removed_sentence = [word
                                     for word in tokenized_sentence
                                         if not word in stopwords]
    # 각 문장별 단어 토큰화 결과 리스트에 추가하기
    X_test.append(stopwords_removed_sentence)
    
print(X_test)

100%|████████████████████████████████████████████████████████████████████████████████| 130/130 [00:10<00:00, 12.59it/s]

[['여보세요', '예', '고객', '님', '롯데', '심사', '팀', '이에요', '예', '예', '예', '아깝다', '좀', '전', '에', '전화', '드리다', '전화', '연결', '이', '안', '되다', '고객', '님', '아', '예', '운전', '중이', '라서', '요', '아', '지금', '운전', '다', '끝내다', '네', '예', '안전', '운전', '하다', '되다', '고객', '님', '그', '지금', '입금', '받다', '계좌', '번호', '예', '카카오', '뱅크', '예', '계좌', '번호', '좀', '불다', '고객', '님', '계좌', '번호', '아깝다', '거기', '에', '있다', '왜', '전화', '하다', '분', '있다', '하다', '내', '가', '이', '거', '휴대폰', '끄다', '휴대폰', '에', '기록', '이', '돼다', '예', '알다', '고객', '님', '네', '예', '고객', '님', '롯데', '캐피탈', '심사', '팀', '전성환', '대리', '고요', '예', '지금', '저희', '쪽', '에서', '신용', '대출', '로', '나가다', '한도', '가', '최대', '억', '인데', '제', '가', '지금', '전산', '상', '에', '억', '으로', '잡히다', '거', '제', '가', '지금', '잘못', '보고', '깜짝', '놀래다', '가다', '예', '전화', '드리다', '거구', '요', '예', '혹시', '고객', '님', '그', '다시', '한번', '제', '가', '지금', '말씀', '을', '여쭈다', '보다', '네', '삐', '고객', '님', '맞다', '네', '네', '예', '고객', '님', '주민', '번호', '앞', '에', '자리', '확인', '좀', '부탁드리다', '삐', '예', '고객', '님', '그', '직장', '주소', '는', '어떻다', '되다

### 정수 인코딩 하기

In [16]:
### 정수 인코딩을 위한 객체 생성
tokenizer = Tokenizer()

In [17]:
### 정수 인코딩하기 : 딕셔너리 형태로 만들어짐
# - 빈도를 자동으로 계산해서 빈도가 높은 순서대로 1부터 값을 부여시킴
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

{'하다': 1, '네': 2, '이': 3, '가': 4, '되다': 5, '을': 6, '에': 7, '있다': 8, '님': 9, '고객': 10, '를': 11, '요': 12, '예': 13, '그렇다': 14, '그': 15, '는': 16, '제': 17, '받다': 18, '은': 19, '거': 20, '아': 21, '으로': 22, '저희': 23, '드리다': 24, '로': 25, '에서': 26, '도': 27, '아니다': 28, '지금': 29, '확인': 30, '없다': 31, '수': 32, '뭐': 33, '대출': 34, '이다': 35, '은행': 36, '가능하다': 37, '알다': 38, '어떻다': 39, '쪽': 40, '안': 41, '계좌': 42, '처리': 43, '만': 44, '좀': 45, '번호': 46, '의': 47, '내': 48, '이제': 49, '저': 50, '해주다': 51, '고': 52, '보험': 53, '진행': 54, '부분': 55, '상환': 56, '서': 57, '말씀': 58, '게': 59, '전화': 60, '맞다': 61, '면': 62, '와': 63, '해드리다': 64, '같다': 65, '께서': 66, '시': 67, '보다': 68, '통장': 69, '다': 70, '가입': 71, '카드': 72, '분': 73, '때문': 74, '가요': 75, '한': 76, '자금': 77, '서류': 78, '죠': 79, '하고': 80, '바로': 81, '보내다': 82, '원': 83, '만원': 84, '안되다': 85, '이체': 86, '필요하다': 87, '까지': 88, '주다': 89, '일단': 90, '발급': 91, '금액': 92, '돈': 93, '접수': 94, '연락': 95, '나오다': 96, '본인': 97, '것': 98, '신청': 99, '일': 100, '적': 101, '이용': 102, '경우': 103, '

### 단어 집합 확인하기

In [18]:
### 희귀단어 기준 정의
threshold = 3

# - 전체 단어 수
total_cnt = len(tokenizer.word_index)

### 희귀단어 기준보다 작은 단어의 개수
rare_cnt = 0

### 훈련 데이터의 전체 단어 빈도수 총합
total_freq = 0

### 희귀단어 기준보다 작은 단어의 빈도수의 총합
rare_freq = 0

In [19]:
### 단어 빈도 개수 확인하기 : 리스트의 튜플 형태로 나옴
tokenizer.word_counts

OrderedDict([('제', 818),
             ('가', 1978),
             ('답답하다', 6),
             ('죽겠네', 1),
             ('지금', 546),
             ('선생님', 34),
             ('본인', 169),
             ('이래', 1),
             ('요', 965),
             ('자기', 8),
             ('네', 2477),
             ('그', 840),
             ('성함', 73),
             ('이', 2377),
             ('아니다', 558),
             ('분', 212),
             ('은행', 440),
             ('을', 1535),
             ('통해', 53),
             ('서', 271),
             ('통화', 69),
             ('하다', 3985),
             ('진짜', 11),
             ('오다', 141),
             ('결과', 56),
             ('는', 830),
             ('보시', 56),
             ('면', 256),
             ('알다', 422),
             ('거', 778),
             ('예요', 145),
             ('어떻다', 419),
             ('사태', 2),
             ('발생', 45),
             ('복잡하다', 6),
             ('돼다', 159),
             ('굉장하다', 11),
             ('경찰', 24),
             ('에서', 594),
     

In [20]:
### 단어와 빈도수의 쌍을 key와 value로 받아서 반복 처리하기
for key, value in tokenizer.word_counts.items() :
    total_freq = total_freq + value
    
    ### 희귀단어 기준보다 작으면
    if value < threshold :
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [21]:
print("단어 집합의 크기 : ", total_cnt)
print("희귀 단어 기준보다 작은 단어수 : ", threshold - 1, rare_cnt)
print("단어 집합에서 희귀 단어의 비율 : ", rare_cnt / total_cnt * 100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 : ", rare_freq / total_freq * 100)

단어 집합의 크기 :  4007
희귀 단어 기준보다 작은 단어수 :  2 1980
단어 집합에서 희귀 단어의 비율 :  49.413526328924384
전체 등장 빈도에서 희귀 단어 등장 빈도 :  2.70894339662248


In [22]:
### 정수인코딩에 적용할 단어의 갯수(사이즈) 결정하기
vocab_size = total_cnt - rare_cnt + 1
print("사이즈 : ", vocab_size)

사이즈 :  2028


In [23]:
### 텍스트 시퀀스(문자)를 정수 시퀀스(정수값)으로 변환
tokenizer = Tokenizer(vocab_size)

### 사이즈만큼의 단어들에 대해서 변환하기
tokenizer.fit_on_texts(X_train)

### 훈련 및 테스트 데이터 변환 적용하기
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(X_train,"\n")
print(X_test)

[[17, 4, 1201, 29, 412, 97, 12, 1043, 4, 2, 15, 229, 3, 28, 15, 73, 3, 28, 36, 6, 292, 57, 239, 1, 1201, 893, 124, 17, 4, 36, 28, 273, 16, 274, 62, 38, 20, 120, 29, 39, 4, 29, 333, 6, 1, 1202, 111, 29, 894, 1202, 535, 26, 38, 1, 12, 50, 27, 423, 7, 127, 11, 1, 17, 4, 11, 1, 17, 4, 1482, 6, 18, 15, 40, 26, 807, 6, 1, 15, 40, 26, 1482, 6, 18, 690, 3, 20, 11, 33, 144, 16, 2, 14, 263, 6, 220, 842, 22, 578, 522, 21, 893, 220, 717, 4, 31, 28, 29, 222, 29, 3, 41, 5, 3, 29, 23, 39, 199, 29, 10, 9, 3, 1, 58, 3, 604, 3, 120, 29, 17, 4, 350, 1203, 58, 499, 12, 29, 244, 3, 15, 244, 3, 28, 29, 350, 244, 6, 106, 109, 58, 1, 68, 239, 11, 1, 101, 3, 31, 12, 239, 1716, 31, 1204, 2, 2, 21, 17, 4, 36, 80, 239, 80, 36, 10, 452, 26, 195, 95, 6, 1, 74, 7, 36, 26, 38, 1, 21, 556, 29, 36, 36, 29, 151, 239, 1, 1, 20, 120, 36, 3, 12, 10, 9, 3, 36, 3, 12, 28, 479, 1717, 40, 36, 3, 12, 36, 152, 1, 40, 36, 3, 12, 46, 4, 152, 152, 1, 15, 285, 15, 68, 15, 623, 624, 60, 1205, 20, 120, 21, 13, 14, 21, 153, 314, 31, 29

In [24]:
### 훈련 및 테스트 종속변수 생성하기
y_train = np.array(train_data["phishing"])
y_test = np.array(test_data["phishing"])
print(y_train, "\n")
print(y_test)
## 데이터 프레임에 있는 데이터를 배열 형태로 받아옴

[1 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 1
 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0
 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0
 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1
 0 1 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 0] 

[1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0
 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0
 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0
 0 1 0 0 1 0 0 0 0 1 0 0

In [25]:
"""
### X_train 데이터에 비어있는 데이터 찾아서 삭제하기
# - 길이가 0인 인덱스 삭제하는 방법으로...

### 각 리스트의 길이가 0인 인덱스 찾기
drop_train = [index for index, sentence in enumerate(X_train)
                          if len(sentence) < 1]

### 찾아낸 인덱스 삭제하기
# - 훈련과 독립변수 및 종속변수 모두 동일한 위치의 인덱스 삭제
X_train = np.delete(X_train, drop_train, axis = 0)
y_train = np.delete(y_train, drop_train, axis = 0)

print(len(X_train), len(y_train))
"""

'\n### X_train 데이터에 비어있는 데이터 찾아서 삭제하기\n# - 길이가 0인 인덱스 삭제하는 방법으로...\n\n### 각 리스트의 길이가 0인 인덱스 찾기\ndrop_train = [index for index, sentence in enumerate(X_train)\n                          if len(sentence) < 1]\n\n### 찾아낸 인덱스 삭제하기\n# - 훈련과 독립변수 및 종속변수 모두 동일한 위치의 인덱스 삭제\nX_train = np.delete(X_train, drop_train, axis = 0)\ny_train = np.delete(y_train, drop_train, axis = 0)\n\nprint(len(X_train), len(y_train))\n'

In [26]:
### 패딩 크기 결정하기
count = 0
max_len = 800
for sentence in X_train :
    if(len(sentence) <= max_len) :
        count = count + 1
print("전체 데이터 중에 길이가 {} 이하인 비율 : {} %".format(
            max_len, (count/len(X_train)) * 100))

전체 데이터 중에 길이가 800 이하인 비율 : 94.85094850948511 %


In [27]:
### 모든 단어의 길이를 max_len으로 맞추기
# - 패딩 처리
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

### LSTM 훈련시키기

In [28]:
### 라이브러리 정의하기
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [29]:
### LSTM 모델 생성 및 계층 추가하기
embedding_dim = 100
hidden_units = 128

model = Sequential()

model.add(Embedding(6392, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation="sigmoid"))

In [30]:
### 콜백 함수 정의
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=3)
mc = ModelCheckpoint("./type1_best_model.h5", monitor="val_acc", mode="max",
                     verbose=1, save_best_only=True)

In [31]:
### 모델 속성 정의하기 : compile
model.compile(optimizer = "adam",
              loss="binary_crossentropy",
              metrics=["acc"])

In [32]:
### 모델 훈련시키기
history = model.fit(X_train, y_train, epochs=50,
                    callbacks=[es, mc], batch_size=64,
                    validation_split=0.2)

Epoch 1/50
5/5 [==============================] - ETA: 0s - loss: 0.6827 - acc: 0.6678
Epoch 1: val_acc improved from -inf to 0.62162, saving model to .\type1_best_model.h5
5/5 [==============================] - 11s 2s/step - loss: 0.6827 - acc: 0.6678 - val_loss: 0.6625 - val_acc: 0.6216
Epoch 2/50
5/5 [==============================] - ETA: 0s - loss: 0.6265 - acc: 0.6780
Epoch 2: val_acc did not improve from 0.62162
5/5 [==============================] - 5s 1s/step - loss: 0.6265 - acc: 0.6780 - val_loss: 0.5843 - val_acc: 0.6216
Epoch 3/50
5/5 [==============================] - ETA: 0s - loss: 0.5509 - acc: 0.6847
Epoch 3: val_acc improved from 0.62162 to 0.68919, saving model to .\type1_best_model.h5
5/5 [==============================] - 6s 1s/step - loss: 0.5509 - acc: 0.6847 - val_loss: 0.5633 - val_acc: 0.6892
Epoch 4/50
5/5 [==============================] - ETA: 0s - loss: 0.5102 - acc: 0.7559
Epoch 4: val_acc improved from 0.68919 to 0.72973, saving model to .\type1_best_mo

In [33]:
### 테스트하기
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 133ms/step - loss: 0.1026 - acc: 0.9615


[0.10262370109558105, 0.9615384340286255]

In [34]:
### 모델 불러들이기
### 훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때
#   저장된 모델인 "naver_best_model.h5"를 로드
loaded_model = load_model("type1_best_model.h5")


### 감성 분류하기

In [35]:
def sentiment_predict(new_sentence) :
    ### 한글 및 공백 이외 모두 제거
    new_sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", new_sentence)
    ### 단어 토큰화 : 어간(표제어) 기준
    new_sentence = okt.morphs(new_sentence, stem=True)
    ### 불용어 처리
    new_sentence = [word for word in new_sentence
                           if not word in stopwords]
    ### 정수 인코딩
    encoded = tokenizer.texts_to_sequences([new_sentence])
    ### 패딩 사이즈 처리
    pad_new = pad_sequences(encoded, maxlen = max_len)

    ### 감성 예측(긍정 or 부정)
    score = float(loaded_model.predict(pad_new))


    if (score > 0.5) :
        print("{}% 확률로 대출사기형 보이스피싱 입니다.".format(score * 100))
    else :
        print("{}% 확률로 대출사기형 보이스피싱이 아닙니다.".format((1 - score) * 100))

In [36]:
sentiment_predict("OOO에서 취급하는 일반 신용 대출, 담보 대출이 아니시구요.', '한국 자산 관리 공사내 정부 정책 자금입니다.', '한국 자산 관리 공사는 여기서 최종..', '최종 승인이 나셔야 여기서 심사가 이루어 지시는데요.', '거기서 최종 승인을 받으신 고객님께서는 대출금은 고객님 사시는곳 근처 OOO 중앙회 내방하셔서 자금 수령이 가능하시고요.', '우선은 저는 저랑 접수를 통해서 서민 금융 통합 지원 센터로 접수가 이관이 되세요.', '거기로 배정을 제가 접수를 이관시켜 드리면은 거기 대출 전문 담당자님이 배정 받으실 겁니다.', '고객님께서 정부 정책 자금 이 정부에서 하는 자금이시구요.', '정부 지원하시는 거고 저희가 저 접수처만 OOO이시고요, 고객님.")

81.03123903274536% 확률로 대출사기형 보이스피싱 입니다.


In [37]:
sentiment_predict("지난여름에 비가 많이와서 차가 반쯤 잠기는 사고가 있었는대요 시민보험이 있더라구요 신청가능한가요        네 보험신청 가능합니다  서울시에 물어봐야되요 구청에 물어봐야되요        서울시와 계약한 보험회사에 문의 하시면 됩니다    네   보험회사가 어딘가요        네 0000보험사 입니다    네   사람이 다친게 아닌 물건 에 대한 보상도 됩니까        네 시민안전보험은 천재지변에 의한 손해도 보상 하고 있습니다    네   제가 개인적으로 든 보험도 있는데 중복 보상 되나요       네 가능합니다  최대 얼마까지 보상되는지요        최대 1000만원까지 입니다    네 알겠습니다      네 좋은하루 되세요 ")

94.22301650047302% 확률로 대출사기형 보이스피싱이 아닙니다.


In [38]:
sentiment_predict("'여보세요', '네 안녕하세요', 'OK 저축은행 신용 법인이고요.', '신용대출 있어도 연체 없으시면 가능하셔서 필요자금 대한 자금 있으세요', '고객님?', '아까... 그게 아니고 지금 따른건가요?', '고객님 저희가 안내를 해드리는 거라서 지금 뭐 필요 자금 있으신 거죠?', '필요자금이요?', '네', '필요자금 있죠', '고객님 연체나 신용에 문제는 없으신 거죠.', '연체나 신용에 문제 없어요.', '사기범 :\u3000있어요？', '없다고요.', '사기범 :아그럼대신 자꾸 다른 소리 가드렸 어요', '고객님,그러면 지금하 시는 일은, 직장다니세요', '개인사업 을하 시는거 에요?', '직장', '사기범 :\u3000사대 보험도 되세요?', '네', '사기범 :\u3000나이\u3000제한만 있는데 몇 년생이시고요?', 'O년생', '사기범 :\u3000O년생이요?', '피해자 :\u3000네', '군대는 다녀오신거죠?', '피해자 :\u3000면제에요', '예?', '피해자 :\u3000면제라고요', '어떤 이유로 면제 받으신 거에요?', '그런거 까지 말씀드려야 되나요?', '아 그래도 면제라고 하시니까 아 그러면 고객님 여기서 일 하시는 몇 개월 정도 되신 거에요', '삼년이요', '사기범 :\u3000삼년 고객님。 뭐 한달안에 서류 넣고 진행한 적 없으신 거죠?', '잘 못들었어요', '다시 한번 말씀해주세요.', '네?', '피해자 :다시한 번말씀해주세요.', '잘못 들 었어요.', '사기범 :\u3000한달 안에 서류 넣고 진행한 적은 없으신 거죠？', '무슨 서류요?', '사기범 :\u3000서류 넣고 대출 받기 위해서 서류 넣고 진행한 적은 없으신 거죠.', '피해자 :\u3000아 예', '예 그러면 되세요.', '월 평균 수입 대략 어느 정도 나오세요?', '피해자 :\u3000삼백만원이요', '삼백이요?', '피해자 :\u3000예', '예 성함은 어떻게 되시고요?', 'OOO', '아유 끊어요'")

81.99601769447327% 확률로 대출사기형 보이스피싱 입니다.


In [39]:
sentiment_predict("대출을 받을 수 있는지 궁금해서요.       대출금리는 담보력, 고객님의 신용도, 대출상품의 종류에 따라 달리 적용될 수 있습니다. 아~ 그럼 여기서는 알 수 없는 거란 말이죠?       네. 그렇습니다. 고객님 그럼 어떻게 해야 하나요?       가까운 영업점에 문의하셔야 정확한?금리를 알수 있습니다. 참고로,?지방자치단체로부터추천을 받거나,?협회 또는 재단으로부터 추천이 있으면 저리로 대출받을?수 있는 상품도 있으니 참고하시기 바랍니다. 그렇군요. 그럼 아파트 담보 대출은 가능한가요?       네. 가능합니다. 아파트 담보 대출에 필요한 서류는 어떻게 되나요?       등기권리증, 인감증명서, 인감도장, 신분증, 주민등록등본,?등기부등본이 필요하고 소득증빙서류 예를 들면 근로소득원천징수영수증 등과 납세증명서 등이 필요합니다.???? 그리고 원래 대출이 있는데 추가로 대출 받을 수 있나요?       대출상담은 민원접수를 통해 진행이 불가합니다.?대출계좌 관리점에내점하셔서?상담을 받으신?후 확인 가능하십니다. 대출을 해줄지 안해줄지는 어떻게 결정하나요?       대출신청인의 사업성, 신청내용의 타당성, 신용도, 재무상태 및 대출금 상환능력 등을 종합적으로검토하여?결정합니다.?그러나?담보물경락자금 지원시에는 당해 담보물 관련 前 차주 또는 前 연대보증인 등 채무관련인에게는?지원하지?않습니다. 네? 담보물경락자금이 뭐에요?       법원 경매에서 낙찰을 받게 되면 매각허가 결정확정 후 1개월 이내에 대금 납부해야 하는데 그 자금을 담보물경락자금이라고 합니다.   아..그렇구나.    혹사 다른 문의사항 있으십니까?     아니요.     네. 감사합니다. 오늘도 행복한 하루되세요.")

91.750767827034% 확률로 대출사기형 보이스피싱이 아닙니다.


In [40]:
import speech_recognition as sr
import pyaudio
# 형태소 분석
from konlpy.tag import Okt
import pandas as pd
okt = Okt()
# 인식할 수 없는 형태의 음원 파일을 변환하기 위한 모듈
from pydub import AudioSegment
# wav 파일 길이를 가져오기
import wave
import contextlib

# 생성된 wav 파일을 지우기 위한 모듈
import os



class voice:    
    def __init__(self, input_file):
        self.file = input_file   # 분석을 원하는 음성 파일
        self.r = sr.Recognizer() # sr의 recognizer 불러옴
        
        self.df = pd.read_csv("500_가중치.csv", encoding='utf-8')             # 전체 형태소 분석 (가중치) 파일 
        self.type_df = pd.read_csv("type_token_가중치.csv", encoding='utf-8') # 범죄 유형 분류 기준 단어 파일
        
        self.cnt = 1        # 보이스피싱 확률 변수
        self.type1_cnt = 1  # 대출사기형 확률
        self.type2_cnt = 1  # 수사기관사칭형 확률        
        self.text = ''      # 음성에서 변환된 텍스트
        self.export_cnt = 0 # 새롭게 wav 파일이 만들어졌는지 여부를 알기 위함
        
    # 음성 파일을 wav 파일로 통일하는 함수
    def to_wav(self):
        try:
            if self.file[self.file.rfind('.')+1:] != 'wav':
                sound = AudioSegment.from_file(self.file) 
                self.file = self.file[:self.file.rfind('.')]+'.wav'
                sound.export(self.file , format="wav")  # 파일을 인식할 수 있도록 파일 형식 변환
                self.export_cnt = 1    # 새롭게 wav 파일이 만들어짐
            
            with contextlib.closing(wave.open(self.file,'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)
                
            self.duration_list = [30]*int(duration/30) + [round(duration%30)]
                
        except:
            print('Error')
            
    # 음성을 텍스트로 변환하는 함수        
    def recognize(self):
        try:
            with sr.AudioFile(self.file) as source:                         # 오디오 파일 불러오기
                for duration in self.duration_list:
                    self.r.adjust_for_ambient_noise(source, duration=0.5)
                    self.r.dynamic_energy_threshold = True
                    audio = self.r.record(source, duration=duration)        
                    try:
                        self.text += self.r.recognize_google(audio_data=audio, language='ko-KR')#집어넣기
                        print('▶ 통화내역 : {}'.format(self.text))  # 통화내역 출력
                    except:
                        None
                
            #print(self.text)
            
            if self.export_cnt == 1:
                if os.path.exists(self.file):
                    os.remove(self.file)               # 텍스트변환 다하면 음성 삭제
                    
        except: 
            print('Error')
    def sentiment_predict(self) :
        ### 한글 및 공백 이외 모두 제거
        self = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", self.text)
        ### 단어 토큰화 : 어간(표제어) 기준
        self = okt.morphs(self, stem=True)
        ### 불용어 처리
        self = [word for word in self
                               if not word in stopwords]
        ### 정수 인코딩
        encoded = tokenizer.texts_to_sequences([self])
        ### 패딩 사이즈 처리
        pad_new = pad_sequences(encoded, maxlen = max_len)

        ### 감성 예측(긍정 or 부정)
        score = float(loaded_model.predict(pad_new))

        if (score > 0.5) :
            print("{}% 확률로 대출사기형 보이스피싱 입니다.".format(score * 100))
        else :
            print("{}% 확률로 대출사기형 보이스피싱이 아닙니다.".format((1 - score) * 100))

                        
    # 결과를 출력하는 함수
    def result(self):
        self.to_wav()
        self.recognize() # 음성 텍스트 변환 함수 호출    
        self.sentiment_predict()

In [41]:
from Voice_Phishing import voice_detection as vd
file_path = 'D:/1조프로젝트/Voice_Phishing/test9.wav' # 파일 경로 담기
vp = voice(file_path)
vp.result()

▶ 통화내역 : 안녕하세요 금융감독원 000 조사관입니다 지금 전세사기 관련하여 조사중 선생님의 우리은행 계좌가 해당 범죄의 대포통장으로 연루되어 현재 동결 조치를 취했습니다 해제를 하려면 본인이 이번 사건과 무관 하다는 걸 입증해 주셔야 하는데 내용 확인을 위해 이제부터 알려드리는 웹 사이트 주소를 통화를 메모를 하셔서 통화를 끊지 말고 접속해 주시기 바랍니다
68.1490808725357% 확률로 대출사기형 보이스피싱이 아닙니다.
